# Webscrapping on tradingview.com

In [ ]:
#import necessary libraries
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import datetime
#import re
import csv
#from io import StringIO
#import json
from IPython.core.interactiveshell import InteractiveShell #multiple outputs for a cell
InteractiveShell.ast_node_interactivity = "all"            #multiple outputs for a cell

In [ ]:
#different categories of interest and url template
categories=['large-cap','active','gainers','losers','most-volatile','overbought','oversold']
choice=categories[2]
url="https://www.tradingview.com/markets/stocks-usa/market-movers-"+"{}"+"/"
response=requests.get(url.format(choice))        #request for a web page
soup=BeautifulSoup(response.text,'html.parser')  #creating the soup object with given page
#date='052821'
# filesuffix=date+'tv.csv'
#file=choice+"tv"+date+".csv"
#csv_file=open(file,"w")
#print("csv-file-name\t",file)
        
    

In [ ]:
#soup

In [ ]:
#find the ticker block
#function to return .csv file for given category
url="https://www.tradingview.com/markets/stocks-usa/market-movers-"+"{}"+"/"
def get_csv(category,filesuffix,root):
    '''
    reutrns the csv file for given category using tradingview.com website for particular day
    '''
    
    response=requests.get(url.format(category))        #request for a web page
    soup=BeautifulSoup(response.text,'html.parser')    #parse the webpage
    
    #header of table
    header=['TICKER','LAST','CHG%','CHG','RATING','VOL','MKT CAP','P/E','EPS(TTM)','EMPLOYEES','SECTOR','INFO'] #columns name
    
    #csv file to store the header and data
    file=root+category+filesuffix
    csv_file=open(file,"w")
    #print("csv-file-name\t",file)
    csv_writer=csv.writer(csv_file)
    csv_writer.writerow(header)

    table=soup.find('table')#table in webpage
    table_body=table.find('tbody',class_="tv-data-table__tbody") #table row
    row_data=table_body.find_all('tr') #row data
    all_data=[]
    web_page='https://www.tradingview.com'
    for tr in row_data:
        tds=tr.find_all('td')
        td_list=[]
        for i,td in enumerate(tds):
            #print(i,td)
            #print("=======================\n")
            if i==0:
                
                #print(30*"x")
                symlink=web_page+td.find('a').get('href')
                ticker=td.find('a').contents[0]
                #print("link\t",symlink)
                #print("TICKER\t",ticker)
                #print()
                #add values to the list
                td_list.append(ticker)
                td_list.append(symlink)
            else:
                #td_list.append(td.text)
                td_list.insert(i,td.text)
        all_data.append(td_list)
    #print(all_data)
    for i in range(len(all_data)):
        #print(all_data[i])
        csv_writer.writerow(all_data[i])
    csv_file.close()
    print("successfully created file: {}".format(file))
    return file


In [ ]:
# root='/home/thakur/cks/tvfiles/'                         #dir to save the file
# date=datetime.date.today().strftime('%m%d%y')            #today
# filesuffix=date+'tv.csv'
# act_file=get_csv('overbought',"oversold.csv")

In [ ]:
#get csv files for all categories
sub=1
root='/home/thakur/cks/tvfiles/'                         #dir to save the file
date=datetime.date.today()
#date=datetime.date.today().strftime('%m%d%y')            #today
#if date is not update
days_sub=datetime.timedelta(days=sub)                         #subtract 1 day
date=date-days_sub
date=date.strftime('%m%d%y')  
filesuffix=date+'tv.csv'
cat=['large-cap','active','gainers','losers','most-volatile','overbought','oversold']
#filesuffix=[root+i for i in cat]
csv_file_list=[]
for category in cat:
    fi=get_csv(category,filesuffix,root)
    csv_file_list.append(fi)
    #print('file\t',fi)
#print("file list\n {}".format(csv_file_list))


In [ ]:
# #get csv files for all categories
# root='/home/thakur/cks/tvfiles/'                         #dir to save the file
# date=datetime.date.today().strftime('%m%d%y')            #today
# filesuffix=date+'tv.csv'
# categories=['large-cap','active','gainers','losers','most-volatile','overbought','oversold']
# categories=[root+i for i in categories]
# csv_file_list=[]
# for category in categories:
#     file=root+category+filesuffix
#     fi=get_csv(category,filesuffix)
#     csv_file_list.append(fi)
#     #print('file\t',fi)
# #print("file list\n {}".format(csv_file_list))


In [ ]:
def make_clickable(val):
    return '<a target ="_blank" href="{}">{}</a>'.format(val,"link")
    #target="_blank" opens the window to new window
    #can use val inplace to "link" to print the exact link
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'green'
    return 'color: %s' % color

In [ ]:
#read the files in pandas
def return_df(file_list,items=10):
    for i in file_list:
        print(30*" "+10*"="+i+10*"=")
        df=pd.read_csv(i)
        display(df.head(items))

In [ ]:
#return_df(file_list=csv_file_list)
#df=return_df(csv_file_list[0])
#df.Styler.format({'INFO':make_clickable})

In [ ]:
for i in csv_file_list:
    df=pd.read_csv(i)
    print(cat[i])
    print(20*'-')
    print(df.head(20))

In [ ]:
#print(categories[6])
#print(csv_file_list)
for i,file in enumerate(csv_file_list):
    print(i,"\t",file)
#df_large=pd.read_csv(csv_file_list[6])
#df_large.head()
df_oversold=pd.read_csv(csv_file_list[4])
df_oversold.sort_values(by=['CHG%'],inplace=True,ascending=True)
#df_oversold
df_oversold=df_oversold.style.format({'INFO':make_clickable})
df_oversold

In [ ]:
#print(categories[6])
#print(csv_file_list)
for i,file in enumerate(csv_file_list):
    print(i,"\t",file)
#df_large=pd.read_csv(csv_file_list[6])
#df_large.head()
df_oversold=pd.read_csv(csv_file_list[4])
#df_oversold
df_oversold=df_oversold.style.format({'INFO':make_clickable})
df_oversold

In [ ]:
# #read all the dataframe and show the heading
# data_frames=["df_"+i for i in categories]
# print("data_frames\t",data_frames)
# for i,category in enumerate(categories):
#     print("========={}==========".format(category))
#     data_frames[i]=pd.read_csv(category+filesuffix)
#     #print(df.head())

In [ ]:
df_large-cap.head()

In [ ]:
%%script false --no-raise-error
#find the ticker block

header=['TICKER','LAST','CHG%','CHG','RATING','VOL','MKT CAP','P/E','EPS(TTM)','EMPLOYEES','SECTOR','INFO'] #columns name
csv_writer=csv.writer(csv_file)
csv_writer.writerow(header)

table=soup.find('table')#table in webpagetable=soup.find('table', class_="tv-data-table tv-screener-table tv-screener-table--fixed")#table in webpage
#print(table)
table_body=table.find('tbody',class_="tv-data-table__tbody") #table row
row_data=table_body.find_all('tr') #row data
#print("row_data\t",row_data)
#print(row_data.prettify())
all_data=[]
# tds=row_data.find_all('td')
# td_list=[]
web_page='https://www.tradingview.com'
for tr in row_data:
    tds=tr.find_all('td')
    td_list=[]
    for i,td in enumerate(tds):
        print(i,td)
        print("=======================\n")
        if i==0:
            #print(td)
            #print(20*"-")
            #print("Content of a tag\n")
            print(30*"x")
            symlink=web_page+td.find('a').get('href')
            ticker=td.find('a').contents[0]
            print("link\t",symlink)
            print("TICKER\t",ticker)
            print()
            #add values to the list
            td_list.append(ticker)
            td_list.append(symlink)
        else:
            #td_list.append(td.text)
            td_list.insert(i,td.text)
    all_data.append(td_list)
print(all_data)
for i in range(len(all_data)):
    print(all_data[i])
    csv_writer.writerow(all_data[i])
csv_file.close()


In [ ]:
soup.title.string

In [ ]:
%%script false --no-raise-error
#find the ticker block
#table=soup.find('table', class_="tv-data-table tv-screener-table tv-screener-table--fixed")#table in webpagetable=soup.find('table', class_="tv-data-table tv-screener-table tv-screener-table--fixed")#table in webpage
csv_file=open("large.csv","w")
header=['TICKER','LAST','CHG%','CHG','RATING','VOL','MKT CAP','P/E','EPS(TTM)','EMPLOYEES','SECTOR'] #columns name
csv_writer=csv.writer(csv_file)
csv_writer.writerow(header)

table=soup.find('table')#table in webpagetable=soup.find('table', class_="tv-data-table tv-screener-table tv-screener-table--fixed")#table in webpage
#print(table)
table_body=table.find('tbody',class_="tv-data-table__tbody") #table row
row_data=table_body.find_all('tr') #row data
#print("row_data\t",row_data)
#print(row_data.prettify())
all_data=[]
# tds=row_data.find_all('td')
# td_list=[]
for tr in row_data:
    tds=tr.find_all('td')
    td_list=[]
    for td in tds:
        td_list.append(td.text)
    all_data.append(td_list)
print(all_data)
for i in range(len(all_data)):
    print(all_data[i])
    csv_writer.writerow(all_data[i])
csv_file.close()
#print(row_data.text)
#table_body=table.get()
#table_body=table.find('tbody')
#table_body=table.find('tbody')
#print('table-body\n',table_body)
# table_data_row=table_body.find('tr')
# table_data=table_data_row.find_all('td') #find all returns the list

# #link="https://finance.yahoo.com"+href
# #print(link)
# tick_info=[]
# for data in table_data[:-1]:
#     #print("data\t",data)
    
#     #print("data text\t",data.text)
    
#     tick_info.append(data.text)
#     #tikc_info.append(link)
# href=table_data[0].find('a').get('href')
# #href=data.find('a').get('href')
# link="https://finance.yahoo.com"+href
# tick_info.append(link)
# print("link\t",link)
# tick_info

In [ ]:
soup.title.string

In [ ]:
soup.div

In [ ]:
soup.find('a').get('href')